<center>
    <h1> ILI-286 - Laboratorio #5 </h1>
    <h2> Sistemas de reacción-difusión </h2>
</center>


| Nombre | Rol | Email |
| :----- | :-- | :---- |
| Marco Rojas | 201073005-0 | marco.rojaso@alumnos.usm.cl |
| Hernán Vargas | 201073009-3 | hernan.vargas@alumnos.usm.cl |

## Tabla de contenidos
* [Introducción](#intro)
* [Desarrollo y analisís de resultados:](#desarrollo)
  1. [Reacción-Difusión y formación de patrones de Turing](#se1)
  2. [Diferencias Finitas en 2D](#se2)
  3. [Método de las líneas](#se3)
  4. [Simulaciones](#se4)
* [Conclusiones](#Conclusiones)
* [Referencias](#Referencias)
* [Anexos](#Anexos) $\leftarrow$ Comenzar ejecución aquí
  * [Métodos implementados](#an1)
  * [Utilidades](#an5)

<div id='intro' />
## Introducción
En este laboratorio se analizará el sistema de reacción-difusión de Alan Turing que propone un modelo
para la formación de pigmentación en animales, particularmente se trabajará con el modelo propuesto por 
FitzHugh-Nagumo donde interactuan dos ecuaciones diferenciales tanto espacialmente como en el tiempo.
Se resolverá el problema por medio de diferencias finitas (tanto con discretizaciones backward como forward)
y por el método de lineas, con estos resultados se estimará la incidencia de los parametros que afectan al
sistema y los límites propios de los algoritmos.

<div id='desarrollo'/>
## Desarrollo y analisís de resultados

<div id='se1'/>
## 1.- Reacción-Difusión y formación de patrones de Turing

El sistema queda definido por:
\begin{align}
    \frac{\partial u}{\partial t} &= 
            D_u \, (\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2}) +
            \lambda \, u - u^3 - \kappa - \sigma v \\
    \tau \, \frac{\partial v}{\partial t} &= 
            D_v \, (\frac{\partial^2 v}{\partial x^2} + 
            \frac{\partial^2 v}{\partial y^2}) + u - v
\end{align}
Se trabajará con condiciones de borde periodicas.

<div id='se2'/>
## 2.- Diferencias finitas en 2D

### a) Propuesta de discretización "Forward"
Utilizando las siguientes aproximaciones:
\begin{align}
    \frac{\partial^2 u}{\partial x^2} &= \frac{u(x+\Delta x, y, t)-2u(x,y,t)+u(x-\Delta x,y,t)}{\Delta x^2}\\
    \frac{\partial^2 u}{\partial y^2} &= \frac{u(x, y+\Delta y, t)-2u(x,y,t)+u(x,y-\Delta y,t)}{\Delta y^2}\\
    \frac{\partial u}{\partial t} &= \frac{u(x,y,t+\Delta t) - u(x,y,t)}{\Delta t}
\end{align}

Análogo para $v(x,y,t)$.

Las ecuaciones quedan como:
\begin{align}
    \frac{u(x,y,t+\Delta t) - u(x,y,t)}{\Delta t} =& \,
        D_u \, \left(
            \frac{u(x+\Delta x, y, t)-2u(x,y,t)+u(x-\Delta x,y,t)}{\Delta x^2} + 
            \frac{u(x, y+\Delta y, t)-2u(x,y,t)+u(x,y-\Delta y,t)}{\Delta y^2} 
        \right)\\
        &+ \lambda \, u(x,y,t) - u(x,y,t)^3 - \kappa - \sigma v(x,y,t)\\
    &\\
    \tau \, \frac{v(x,y,t+\Delta t) - v(x,y,t)}{\Delta t} =&\,
        D_v \, \left(
            \frac{v(x+\Delta x,y,t)-2v(x,y,t)+v(x-\Delta x,y,t)}{\Delta x^2} + 
            \frac{v(x,y+\Delta y,t)-2v(x,y,t)+v(x,y-\Delta y,t)}{\Delta y^2} 
        \right)\\
        &+ u(x,y,t) - v(x,y,t)
\end{align}

Ahora si reemplazamos $w_{i,j,k} = u(x,y,t)$ y $ b_{i,j,k} = v(x,y,t) $, donde:
\begin{matrix}
    x= x_0+i \,\Delta x &\Longrightarrow &i= \frac{x-x_0}{\Delta x}\\
    y= y_0+j \,\Delta y &\Longrightarrow &j= \frac{y-y_0}{\Delta y}\\
    t= t_0+k \,\Delta t &\Longrightarrow &k= \frac{t-t_0}{\Delta t}
\end{matrix}

Finalmente, obtenemos:

\begin{align}
    \frac{w_{i,j,k+1} - w_{i,j,k}}{\Delta t} &=
        D_u  \, \left(
            \frac{w_{i+1, j, k}-2w_{i,j,k}+w_{i-1,j,k}}{\Delta x^2} +
            \frac{w_{i, j+1, k}-2w_{i,j,k}+w_{i,j-1,k}}{\Delta y^2} \right)
        + \lambda \, w_{i,j,k} - w_{i,j,k}^3 - \kappa - \sigma b_{i,j,k} \\
    \tau \, \frac{b_{i,j,k+1} - b_{i,j,k}}{\Delta t} &=
        D_v \, \left(
            \frac{b_{i+1,j,k}-2b_{i,j,k}+b_{i-1,j,k}}{\Delta x^2} + 
            \frac{b_{i,j+1,k}-2b_{i,j,k}+b_{i,j-1,k}}{\Delta y^2} \right)
        + w_{i,j,k} - b_{i,j,k}
\end{align}

Despejando para el tiempo $k+1$ nos queda:
\begin{align}
    w_{i,j,k+1} &=
        \Delta t \left(D_u  \, \left(
            \frac{w_{i+1, j, k}-2w_{i,j,k}+w_{i-1,j,k}}{\Delta x^2} + 
            \frac{w_{i, j+1, k}-2w_{i,j,k}+w_{i,j-1,k}}{\Delta y^2} \right)
        + \lambda \, w_{i,j,k} - w_{i,j,k}^3 - \kappa - \sigma b_{i,j,k} \right ) + w_{i,j,k} \\
    b_{i,j,k+1} &= 
        \frac{\Delta t}{\tau} \left(D_v \, \left(
            \frac{b_{i+1,j,k}-2b_{i,j,k}+b_{i-1,j,k}}{\Delta x^2} +
            \frac{b_{i,j+1,k}-2b_{i,j,k}+b_{i,j-1,k}}{\Delta y^2} \right)
        + w_{i,j,k} - b_{i,j,k} \right ) + b_{i,j,k}
\end{align}

El error de aproximación es de: $O(\Delta t) + O(\Delta x^2) + O(\Delta y^2)$

### b) Implementación de esquema explícito "Forward"

In [ ]:
def solve_forward(u0, v0, dx, dy, dt, Tf, Du, Dv, lamb, tau, sigma, kappa):
    Ny = u0.shape[0];
    Nx = u0.shape[1];
    Nt = (int)(np.ceil(Tf/dt))+1;
    w = np.zeros([Nt, Nx, Ny])
    b = np.zeros([Nt, Nx, Ny])
    w[0,:,:] = u0;
    b[0,:,:] = v0;
    for k in range (Nt-1):
        u_xx = (np.roll(w[k,:,:], 1, axis=1) - 2 * w[k,:,:] + np.roll(w[k,:,:], -1, axis=1))/dx**2;
        u_yy = (np.roll(w[k,:,:], 1, axis=0) - 2 * w[k,:,:] + np.roll(w[k,:,:], -1, axis=0))/dy**2;
        u_laplaciano = u_xx + u_yy;
        
        v_xx = (np.roll(b[k, :, :], 1, axis=1) - 2 * b[k,:,:] + np.roll(b[k,:,:], -1, axis=1))/dx**2;
        v_yy = (np.roll(b[k, :, :], 1, axis=0) - 2 * b[k,:,:] + np.roll(b[k,:,:], -1, axis=0))/dy**2;
        v_laplaciano = v_xx + v_yy
        
        w[k+1,:,:] = dt * (Du * u_laplaciano + lamb * w[k,:,:] - np.power(w[k,:,:],3) - kappa - sigma * b[k,:,:]) + w[k,:,:];
        
        b[k+1,:,:] = (dt/tau) * (Dv * v_laplaciano + w[k,:,:] - b[k,:,:]) + b[k,:,:];
    return w;

### c) Propuesta de discretización "Backward"
Aproximaciones:
\begin{align}
    \frac{\partial^2 u}{\partial x^2} &= \frac{u(x+\Delta x, y, t)-2u(x,y,t)+u(x-\Delta x,y,t)}{\Delta x^2}\\
    \frac{\partial^2 u}{\partial y^2} &= \frac{u(x, y+\Delta y, t)-2u(x,y,t)+u(x,y-\Delta y,t)}{\Delta y^2}\\
    \frac{\partial u}{\partial t} &= \frac{u(x,y,t) - u(x,y,t-\Delta t)}{\Delta t}
\end{align}

Análogo para $v(x,y,t)$.

Las ecuaciones quedan como:
\begin{align}
    \frac{u(x,y,t) - u(x,y,t - \Delta t)}{\Delta t} =&\,
        D_u \, \left(
            \frac{u(x+\Delta x, y, t)-2u(x,y,t)+u(x-\Delta x,y,t)}{\Delta x^2} + 
            \frac{u(x, y+\Delta y, t)-2u(x,y,t)+u(x,y-\Delta y,t)}{\Delta y^2} \right)\\
        &+ \lambda \, u(x,y,t) - u(x,y,t)^3 - \kappa - \sigma v(x,y,t)\\
        &\\
    \tau \, \frac{v(x,y,t) - v(x,y,t-\Delta t)}{\Delta t} =&\,
        D_v \, \left(
            \frac{v(x+\Delta x,y,t)-2v(x,y,t)+v(x-\Delta x,y,t)}{\Delta x^2} +
            \frac{v(x,y+\Delta y,t)-2v(x,y,t)+v(x,y-\Delta y,t)}{\Delta y^2} \right)\\
        &+ u(x,y,t) - v(x,y,t)
\end{align}

Ahora si reemplazamos $w_{i,j,k} = u(x,y,t)$ y $ b_{i,j,k} = v(x,y,t) $, donde:
\begin{matrix}
    x= x_0+i \,\Delta x &\Longrightarrow &i= \frac{x-x_0}{\Delta x}\\
    y= y_0+j \,\Delta y &\Longrightarrow &j= \frac{y-y_0}{\Delta y}\\
    t= t_0+k \,\Delta t &\Longrightarrow &k= \frac{t-t_0}{\Delta t}
\end{matrix}

Finalmente, obtenemos:
\begin{align}
    \frac{w_{i,j,k} - w_{i,j,k-1}}{\Delta t} &=
        D_u  \, \left(
            \frac{w_{i+1, j, k}-2w_{i,j,k}+w_{i-1,j,k}}{\Delta x^2} +
            \frac{w_{i, j+1, k}-2w_{i,j,k}+w_{i,j-1,k}}{\Delta y^2} \right)
        + \lambda \, w_{i,j,k} - w_{i,j,k}^3 - \kappa - \sigma b_{i,j,k}\\
    \tau \, \frac{b_{i,j,k} - b_{i,j,k-1}}{\Delta t} &=
        D_v \, \left(
            \frac{b_{i+1,j,k}-2b_{i,j,k}+b_{i-1,j,k}}{\Delta x^2} + 
            \frac{b_{i,j+1,k}-2b_{i,j,k}+b_{i,j-1,k}}{\Delta y^2} \right)
        + w_{i,j,k} - b_{i,j,k}
\end{align}

Cambiando $k$ por $k+1$:
\begin{align}
    \frac{w_{i,j,k+1} - w_{i,j,k}}{\Delta t} &=
        D_u  \, \left(
            \frac{w_{i+1, j, k+1}-2w_{i,j,k+1}+w_{i-1,j,k+1}}{\Delta x^2} +
            \frac{w_{i, j+1, k+1}-2w_{i,j,k+1}+w_{i,j-1,k+1}}{\Delta y^2} \right)
        + \lambda \, w_{i,j,k+1} - w_{i,j,k+1}^3 - \kappa - \sigma b_{i,j,k+1}\\
    \tau \, \frac{b_{i,j,k+1} - b_{i,j,k}}{\Delta t} &=
        D_v \, \left(
            \frac{b_{i+1,j,k+1}-2b_{i,j,k+1}+b_{i-1,j,k+1}}{\Delta x^2} +
            \frac{b_{i,j+1,k+1}-2b_{i,j,k+1}+b_{i,j-1,k+1}}{\Delta y^2} \right)
        + w_{i,j,k+1} - b_{i,j,k+1}
\end{align}        

Despejando para el tiempo $k+1$:

\begin{align}
    w_{i,j,k} &= 
        w_{i,j,k+1} - \Delta t \left ( D_u  \, \left(
            \frac{w_{i+1, j, k+1}-2w_{i,j,k+1}+w_{i-1,j,k+1}}{\Delta x^2} +
            \frac{w_{i, j+1, k+1}-2w_{i,j,k+1}+w_{i,j-1,k+1}}{\Delta y^2} \right)
        + \lambda \, w_{i,j,k+1} - w_{i,j,k+1}^3 - \kappa - \sigma b_{i,j,k+1} \right )\\
    b_{i,j,k} &= 
        b_{i,j,k+1} - \frac{\Delta t}{\tau} \left( D_v \, \left(
            \frac{b_{i+1,j,k+1}-2b_{i,j,k+1}+b_{i-1,j,k+1}}{\Delta x^2} +
            \frac{b_{i,j+1,k+1}-2b_{i,j,k+1}+b_{i,j-1,k+1}}{\Delta y^2} \right)
        + w_{i,j,k+1} - b_{i,j,k+1} \right )
\end{align}

Agrupando incógnitas en la primera ecuación
\begin{align}
    w_{i,j,k} =&\,
        w_{i,j,k+1} - \frac{\Delta t \, D_u \, w_{i+1, j, k+1}}{\Delta x^2} + 
        2\frac{\Delta t \, D_u \, w_{i,j,k+1}}{\Delta x^2} - 
        \frac{\Delta t \, D_u \, w_{i-1,j,k+1}}{\Delta x^2} - 
        \frac{\Delta t \, D_u \, w_{i, j+1, k+1}}{\Delta y^2} +
        2 \frac{\Delta t \, D_u \, w_{i,j,k+1}}{\Delta y^2} -
        \frac{\Delta t \, D_u \, w_{i,j-1,k+1}}{\Delta y^2}\\
        &- \Delta t \, \lambda \, w_{i,j,k+1} + \Delta t \, w_{i,j,k+1}^3 + 
        \Delta t \, \kappa + \Delta t \, \sigma b_{i,j,k+1}\\
    &\\
     w_{i,j,k} - \Delta t \, \kappa =&\,
         (1+ 2\frac{\Delta t \, D_u}{\Delta x^2} + 
         2 \frac{\Delta t \, D_u}{\Delta y^2 } - 
         \Delta t \, \lambda + \Delta t \, w_{i,j,k+1}^2) \, w_{i,j,k+1} -
         \frac{\Delta t \, D_u \, w_{i+1, j, k+1}}{\Delta x^2} -
         \frac{\Delta t \, D_u \, w_{i-1,j,k+1}}{\Delta x^2} -
         \frac{\Delta t \, D_u \, w_{i, j+1, k+1}}{\Delta y^2}
         - \frac{\Delta t \, D_u \, w_{i,j-1,k+1}}{\Delta y^2} \\
         &+ \Delta t \, \sigma b_{i,j,k+1}\\
\end{align}

Cambiando $ \delta_u = \Delta t \, D_u $:

$$ (1+ 2\frac{\delta_u}{\Delta x^2} + 2 \frac{\delta_u}{\Delta y^2 } - \frac{\delta_u}{D_u}  \, \lambda + \frac{\delta_u}{D_u} \, w_{i,j,k+1}^2) \, w_{i,j,k+1} - \frac{\delta_u \, w_{i+1, j, k+1}}{\Delta x^2} - \frac{\delta_u \, w_{i-1,j,k+1}}{\Delta x^2} - \frac{\delta_u \, w_{i, j+1, k+1}}{\Delta y^2} - \frac{\delta_u \, w_{i,j-1,k+1}}{\Delta y^2} + \frac{\delta_u}{D_u} \, \sigma b_{i,j,k+1} = w_{i,j,k} - \frac{\delta_u}{D_u} \, \kappa $$


Agrupando incógnitas en la segunda ecuación
\begin{align}
     b_{i,j,k} &=
         b_{i,j,k+1} - \frac{\Delta t \, D_v \, b_{i+1,j,k+1}}{\Delta x^2 \, \tau} +
         2 \frac{\Delta t \, D_v \, b_{i,j,k+1}}{\Delta x^2 \, \tau} -
         \frac{\Delta t \, D_v \, b_{i-1,j,k+1}}{\Delta x^2 \, \tau} -
         \frac{\Delta t \, D_v \, b_{i,j+1,k+1}}{\Delta y^2 \, \tau} +
         2 \frac{\Delta t \, D_v \, b_{i,j,k+1}}{\Delta y^2 \, \tau} -
         \frac{\Delta t \, D_v \, b_{i,j-1,k+1}}{\Delta y^2 \, \tau} -
         \frac{\Delta t}{\tau} \, w_{i,j,k+1} + \frac{\Delta t \, b_{i,j,k+1}}{\tau}\\
     b_{i,j,k} &=
         (1+ 2 \frac{\Delta t \, D_v}{\Delta x^2 \, \tau} +
         2 \frac{\Delta t \, D_v}{\Delta y^2 \, \tau} +
         \frac{\Delta t}{\tau}) \, b_{i,j,k+1} -
         \frac{\Delta t \, D_v}{\Delta x^2 \, \tau} \, b_{i+1,j,k+1} -
         \frac{\Delta t \, D_v}{\Delta x^2 \, \tau} \, b_{i-1,j,k+1} -
         \frac{\Delta t \, D_v}{\Delta y^2 \, \tau} \, b_{i,j+1,k+1} -
         \frac{\Delta t \, D_v}{\Delta y^2 \, \tau} \, b_{i,j-1,k+1} -
         \frac{\Delta t}{\tau} \, w_{i,j,k+1}
\end{align}

Cambiando $ \delta_v = \displaystyle \frac{\Delta t \, D_v}{\tau} $

$$ (1+ 2 \frac{\delta_v}{\Delta x^2} + 2 \frac{\delta_v}{\Delta y^2} + \frac{\delta_v}{D_v}) \, b_{i,j,k+1} - \frac{\delta_v}{\Delta x^2} \, b_{i+1,j,k+1}   - \frac{\delta_v}{\Delta x^2} \, b_{i-1,j,k+1} - \frac{\delta_v}{\Delta y^2} \, b_{i,j+1,k+1}  - \frac{\delta_v}{\Delta y^2} \, b_{i,j-1,k+1} - \frac{\delta_v}{D_v} \, w_{i,j,k+1} = b_{i,j,k} $$

El error de aproximación es de: $O(\Delta t) + O(\Delta x^2) + O(\Delta y^2)$

En resumen, usando $ \delta_u = \displaystyle \Delta t \, D_u $ y $ \delta_v = \displaystyle \frac{\Delta t \, D_v}{\tau} $ nos queda finalmente el siguiente sistema de ecuaciones no-lineales:

\begin{align}
    (1+ 2\frac{\delta_u}{\Delta x^2} + 2 \frac{\delta_u}{\Delta y^2 } -
    \frac{\delta_u}{D_u}  \, \lambda + \frac{\delta_u}{D_u} \, w_{i,j,k+1}^2) \, w_{i,j,k+1} -
    \frac{\delta_u \, w_{i+1, j, k+1}}{\Delta x^2} - \frac{\delta_u \, w_{i-1,j,k+1}}{\Delta x^2} -
    \frac{\delta_u \, w_{i, j+1, k+1}}{\Delta y^2} - \frac{\delta_u \, w_{i,j-1,k+1}}{\Delta y^2} +
    \frac{\delta_u}{D_u} \, \sigma b_{i,j,k+1} &= w_{i,j,k} - \frac{\delta_u}{D_u} \, \kappa \\
    (1+ 2 \frac{\delta_v}{\Delta x^2} + 2 \frac{\delta_v}{\Delta y^2} +
    \frac{\delta_v}{D_v}) \, b_{i,j,k+1} - \frac{\delta_v}{\Delta x^2} \, b_{i+1,j,k+1} -
    \frac{\delta_v}{\Delta x^2} \, b_{i-1,j,k+1} - \frac{\delta_v}{\Delta y^2} \, b_{i,j+1,k+1} -
    \frac{\delta_v}{\Delta y^2} \, b_{i,j-1,k+1} - \frac{\delta_v}{D_v} \, w_{i,j,k+1} &= b_{i,j,k}
\end{align}

### d) Implementación de esquema implícito "Backward"

In [ ]:
def solve_backward(u0, v0, dx, dy, dt, Tf, Du, Dv, lamb, tau, sigma, kappa):
    Ny = u0.shape[0];
    Nx = u0.shape[1];
    Nt = (int)(np.ceil(Tf/dt));
    w = np.zeros([Nt, Nx, Ny])
    b = np.zeros([Nt, Nx, Ny])
    w[0,:,:] = u0;
    b[0,:,:] = v0;
    
    delta_u = dt*Du;
    delta_v = dt*Dv/tau;
    cons1 = 1+2*delta_u / dx**2 +2 * delta_u/dy**2-delta_u*lamb/Du;
    cons2 = 1+2*delta_v/dx**2+2*delta_v/dy**2+delta_v/Dv;
    for k in range (Nt-1):
        print("Tiempo k=")
        print(k)
        def func(wbk):
            wbk = wbk.reshape(2*Nx, Ny);
            wk = wbk[0:Nx,:];
            bk = wbk[Nx:2*Nx,:];
            return np.array([np.multiply( cons1 + (delta_u*np.power(wk[:,:],2))/Du, wk[:,:])
                    -delta_u*np.roll(wk[:,:], -1, axis=1)/dx**2-delta_u*np.roll(wk[:,:], 1, axis=1)
                         /dx**2-delta_u*np.roll(wk[:,:], -1, axis=0)/dy**2-delta_u*np.roll(wk[:,:], 1, axis=0)/dy**2
                    +delta_u*sigma*bk[:,:]/Du + delta_u*kappa/Du - w[k,:,:]
                     ,
                     cons2*bk[:,:]-delta_v*np.roll(bk[:,:], -1, axis=1)/dx**2
                     -delta_v*np.roll(bk[:,:], 1, axis=1)/dx**2-delta_v*np.roll(bk[:,:], -1, axis=0)
                         /dy**2-delta_v*np.roll(bk[:,:], 1, axis=0)/dy**2-delta_v*wk[:,:]/Dv
                     -b[k,:,:]
                    ]).ravel();
        wk0 = np.ones([Nx, Ny]);
        bk0 = wk0;
        temp = sp.optimize.fsolve(func, np.array([wk0, bk0]).ravel()).reshape(2,Nx,Ny);
        w[k+1,:,:] = temp[0,:,:];
        b[k+1,:,:] = temp[1,:,:];
    return w;

<div id='se3'/>
## 3.- Método de las líneas

### a) Discretización y formulación del sistema

Tenemos:
\begin{align}
    \frac{\partial u}{\partial t} &= 
            D_u \, (\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2}) +
            \lambda \, u - u^3 - \kappa - \sigma v \\
    \tau \, \frac{\partial v}{\partial t} &= 
            D_v \, (\frac{\partial^2 v}{\partial x^2} + 
            \frac{\partial^2 v}{\partial y^2}) + u - v
\end{align}

utilizando diferencias centradas para discretizar el dominio espacial nos queda:

\begin{align}
    \frac{\partial u(x_i,y_j,t)}{\partial t} =&\,
            D_u \left(
                \frac{u(x_{i+1},y_j,t) - 2 u(x_i,y_j,t) + u(x_{i-1},y_j,t)}{\Delta x^2} + 
                \frac{u(x_i,y_{j+1},t) - 2 u(x_i,y_j,t) + u(x_i,y_{j-1},t)}{\Delta y^2}
            \right)\\
            & + \lambda u(x_i,y_j,t) - (u(x_i,y_j,t))^3 - \kappa - \sigma v(x_i,y_j,t) \\
    &\\
    \tau \, \frac{\partial v(x_i,y_j,t)}{\partial t} =&\,
            D_v \left(
                \frac{v(x_{i+1},y_j,t) - 2 v(x_i,y_j,t) + v(x_{i-1},y_j,t)}{\Delta x^2} + 
                \frac{v(x_i,y_{j+1},t) - 2 v(x_i,y_j,t) + v(x_i,y_{j-1},t)}{\Delta y^2}
            \right)\\
            & + u(x_i,y_j,t) - v(x_i,y_j,t)
\end{align}

como podemos notar que ahora el problema diferencial es solo dependiente del tiempo. Hacemos el cambio de variable $u(x_i, y_j, y) = w_{i,j}(t)$ y $v(x_i, y_j, y) = b_{i,j}(t)$.
Así, el sistema dinámico acoplado resultante queda determinado por las siguientes ecuaciones:

\begin{align}
    \dot{w_{ij}}&=
        D_u \left( 
            \frac{w_{i+1,j} - 2 w_{i,j} + w_{i-1,j}}{\Delta x^2} + 
            \frac{w_{i,j+1} - 2 w_{i,j} + w_{i,j-1}}{\Delta y^2} \right) 
        + \lambda w_{i,j} - w^3_{i,j} - \kappa - \sigma b_{i,j} \\
    \dot{b_{i,j}}&= 
        \frac{D_v}{\tau} \left(
            \frac{b_{i+1,j} - 2 b_{i,j} + b_{i-1,j}}{\Delta x^2} + 
            \frac{b_{i,j+1} - 2 b_{i,j} + b_{i,j-1}}{\Delta y^2} \right) 
        + \frac{w_{i,j} - b_{i,j}}{\tau}
\end{align}

Ordenando convenientemente tenemos:
\begin{align}
    \dot{w_{i,j}} &=
        \frac{D_u}{\Delta x^2}(w_{i-1,j}+w_{i+1,j}) + 
        \frac{D_u}{\Delta y^2}(w_{i,j-1}+w_{i,j+1}) +
        (\lambda - \frac{2D_u(\Delta x^2 +\Delta y^2)}{\Delta x^2 \Delta y^2})w_{i,j} - w_{i,j}^3
        -\kappa -\sigma b_{i,j} \\
    \dot{b_{i,j}} &=
        \frac{D_v}{\tau \Delta x^2}(b_{i-1,j}+b_{i+1,j}) +
        \frac{D_v}{\tau \Delta y^2}(b_{i,j-1}+b_{i,j+1}) -
        \left(\frac{2D_v(\Delta x^2 +\Delta y^2)}{\tau\Delta x^2 \Delta y^2} + 
        \frac{1}{\tau}\right)b_{i,j} + \frac{w_{i,j}}{\tau}
\end{align}

### b) Implementación

In [ ]:
def solve_mol(u0, v0, dx, dy, dt, Tf, Du, Dv, lamb, tau, sigma, kappa, method='RK4'):
    if method not in ['RK4', 'RK2']:
        print("El método debe ser 'RK2' o 'RK4'")
        return None
    Ny, Nx, Nt = u0.shape[0], u0.shape[1], int(np.ceil(Tf/dt))
    w = np.zeros([Nt, Nx, Ny])
    b = np.zeros([Nt, Nx, Ny])
    w[0,:,:] = u0;
    b[0,:,:] = v0;
    #Algunos calculos que no varian con el tiempo:
    h2  = dt/2
    dx2, dy2 = dx**2, dy**2
    Dux, Duy = Du/dx2, Du/dy2
    ct1 = lamb - (2*Du*(dx2 + dy2))/(dx2*dy2)
    Dvx, Dvy = Dv/(tau*dx2), Dv/(tau*dy2)
    ct2 = (2*Dv*(dx2 + dy2))/(dx2*dy2) + 1/tau
        
    #Como las ecuaciones son autonomas se utilizará t como subindice:
    def U(t, xy): #t:subindice del tiempo, xy:matriz de 2 dimenciones con los valores en x e y.
        return Dux*(np.roll(xy, -1, axis=1) + np.roll(xy, 1, axis=1)) +\
               Duy*(np.roll(xy, -1, axis=0) + np.roll(xy, 1, axis=0)) +\
               ct1*xy - np.power(xy,3) - sigma*b[t,:,:] - kappa
    def V(t, xy): #igual que la anterior pero para V
        return Dvx*(np.roll(xy, -1, axis=1) + np.roll(xy, 1, axis=1)) +\
               Dvy*(np.roll(xy, -1, axis=0) + np.roll(xy, 1, axis=0)) -\
               ct2*xy + w[t,:,:]/tau
    #Ignorando los cambios en t a proposito (no cambia nada por que la ec es autonoma).
    if method=='RK2':
        for k in range(Nt-1):
            ku1 = w[k,:,:] + h2*U(k, w[k,:,:])
            w[k+1,:,:] = w[k,:,:] + dt*U(k, ku1)
            kv1 = b[k,:,:] + h2*V(k, b[k,:,:])
            b[k+1,:,:] = b[k,:,:] + dt*V(k, kv1)
    else:
        for k in range(Nt-1):
            ku1 = U(k, w[k,:,:])
            ku2 = U(k, w[k,:,:] + h2*ku1)
            ku3 = U(k, w[k,:,:] + h2*ku2)
            ku4 = U(k, w[k,:,:] + dt*ku3)
            w[k+1,:,:] = w[k,:,:] + (dt/6)*(ku1 + 2*(ku2+ku3) + ku4)
            kv1 = V(k, b[k,:,:])
            kv2 = V(k, b[k,:,:] + h2*kv1)
            kv3 = V(k, b[k,:,:] + h2*kv2)
            kv4 = V(k, b[k,:,:] + dt*kv3)
            b[k+1,:,:] = b[k,:,:] + (dt/6)*(kv1 + 2*(kv2+kv3) + kv4)
    return w

<div id='se4'\>
## 4.- Simulaciones
Se establecen las variables a usar:

In [ ]:
#Generando puntos de la malla aleatoria
Nx, Ny = 50, 50
u0 = np.random.rand(Nx,Ny)
v0 = np.random.rand(Nx,Ny)
dx, dy = 1/(Nx-1), 1/(Ny-1)
tf   = 10

### a) Significado de los parametros
Para determinar el significado se harán las siguientes pruebas:

In [ ]:
#test de parametros para un tiempo tf=3, son varias pruebas asi que la ejecución es lenta!!
params = [
    (u0, v0, dx, dy, dx**2, 3, 1e-2, 1e-3, 1, 0.2, 1, -4e-3),
    (u0, v0, dx, dy, dx**2, 3, 1e-4, 1e-3, 1, 0.2, 1, -4e-3),
    (u0, v0, dx, dy, dx**2, 3, 1e-8, 1e-3, 1, 0.2, 1, -4e-3),
    (u0, v0, dx, dy, dx**2, 3, 1e-3, 1e-2, 1, 0.2, 1, -4e-3),
    (u0, v0, dx, dy, dx**2, 3, 1e-3, 1e-8, 1, 0.2, 1, -4e-3),
    (u0, v0, dx, dy, dx**2, 3, 1e-3, 1e-14, 1, 0.2, 1, -4e-3),
    (u0, v0, dx, dy, dx**2, 3, 1e-3, 1e-3, 0.5, 0.2, 1, -4e-3),
    (u0, v0, dx, dy, dx**2, 3, 1e-3, 1e-3, 0.2, 0.2, 1, -4e-3),
    (u0, v0, dx, dy, dx**2, 3, 1e-3, 1e-3, 1.5, 0.2, 1, -4e-3),
    (u0, v0, dx, dy, dx**2, 3, 1e-3, 1e-3, 3, 0.2, 1, -4e-3),
    (u0, v0, dx, dy, dx**2, 3, 1e-3, 1e-3, 1, 0.002, 1, -4e-3),
    (u0, v0, dx, dy, dx**2, 3, 1e-3, 1e-3, 1, 0.8, 1, -4e-3),
    (u0, v0, dx, dy, dx**2, 3, 1e-3, 1e-3, 1, 0.2, 0.3, -4e-3),
    (u0, v0, dx, dy, dx**2, 3, 1e-3, 1e-3, 1, 0.2, 0.01, -4e-3),
    (u0, v0, dx, dy, dx**2, 3, 1e-3, 1e-3, 1, 0.2, 1.5, -4e-3),
    (u0, v0, dx, dy, dx**2, 3, 1e-3, 1e-3, 1, 0.2, 4, -4e-3),
    (u0, v0, dx, dy, dx**2, 3, 1e-3, 1e-3, 1, 0.2, 10, -4e-3),
    (u0, v0, dx, dy, dx**2, 3, 1e-3, 1e-3, 1, 0.2, 1, +4e-3),
    (u0, v0, dx, dy, dx**2, 3, 1e-3, 1e-3, 1, 0.2, 1, 0),
    (u0, v0, dx, dy, dx**2, 3, 1e-3, 1e-3, 1, 0.2, 1, 3),
    (u0, v0, dx, dy, dx**2, 3, 1e-3, 1e-3, 1, 0.2, 1, -5)
]
rs = [None]*len(params)
#for i in range(len(params)):
#    rs[i] = solve_forward(*params[i])

In [ ]:
#animate_pattern(rs[0])

Por lo que se puede ver tenemos:
* $D_u$: mientras más cercano a $0$ el patrón es menos difuso, con $1e-2$ se forman pocas manchas, mientras el número disminuye se ven más manchas y más marcadas, se llegó a $1e-8$ donde se diferencia claramente las secciones azules de las verdes.
* $D_v$: (Con $D_u$ constante) El color azul queda más concentrado cuando el valor es alto (en $1e-2$) mientras que con valores cercanos a $0$ ($1e-14$) se expande más.
* $\lambda$: No se ven cambios cuando el valor es menor a $1$, pero cuando es mayor los colores cambian rapidamente, la imagen se vuelve difusa y mientras avanza el tiempo puede pasar de un color a otro rapidamente.
* $\tau$: Determina cuanto cambia la imagen. Con un valor cercano a $0$ la imagen casi no cambia, cuando se aumenta la imagen tiende a hacerse borrosa.
* $\sigma$: Cuando el valor es menor a $1$ no hay muchos cambios, cuando aumenta los colores empiezan a cambiar mientras pasa el tiempo, con un valor de $10$, la muestra cambia de colores varias veces.
* $\kappa$: Al igual que el anterior, con valores cercanos a $0$ no hay mucho cambio, cuando la magnitud del valor aumenta ocurren cambios bruscos en el tiempo. La imagen termina con solo un par de puntos de cada color bien difuminados.

### b) Ejecución de las implementaciones

In [ ]:
#Parametros suministrados.
Du = 3e-4
Dv = 4e-3
lamb = 1.0
tau  = 0.2
sigm = 1.0
kapp = -0.004

In [ ]:
A = solve_forward(u0, v0, dx, dy, dx**2, tf, Du, Dv, lamb, tau, sigm, kapp)

In [ ]:
B = solve_backward(u0, v0, dx, dy, 2, tf, Du, Dv, lamb, tau, sigm, kapp) #Cuidado: Con dt = 2 tardá 24 min aprox.

In [ ]:
C = solve_mol(u0, v0, dx, dy, dx**2, tf, Du, Dv, lamb, tau, sigm, kapp,'RK2')

In [ ]:
D = solve_mol(u0, v0, dx, dy, 0.5, tf, Du, Dv, lamb, tau, sigm, kapp,'RK4')

In [ ]:
animate_pattern(A)

In [ ]:
animate_pattern(B)

### c) Simulaciones con diferentes parametros
Se hicieron las simulaciones con `solve_forward` igual que en a). Se harán 3 simulaciones como siguen:

In [ ]:
#test de parametros para un tiempo tf=10
params = [
    (u0, v0, dx, dy, dx**2, tf, 3e-2, 4e-3, 1, 0.2, 1, -4e-3),  #Se espera que la imagen final sea completamente difusa
    (u0, v0, dx, dy, dx**2, tf, 3e-4, 4e-3, 1, 0.2, 4, -10),    #Se espera que cambie en el tiempo varias veces
    (u0, v0, dx, dy, dx**2, tf, 3e-15, 4e-3, 1, 0.2, 1, -4e-3)  #Se espera que termine con colores bien definidos
]
rs = [None]*len(params)

for i in range(len(params)):
    rs[i] = solve_forward(*params[i])

In [ ]:
animate_pattern(rs[2])

Como podemos notar los resultados apoyan lo escrito en a)

## Conclusiones
En este trabajo podemos notar que para la resolución de sistemas de ecuaciones parciales se pueden plantear diferentes
modelos adaptandonos a la situación, cada uno con ventajas y desventajas. No hay una formula fija para solucionar 
los problemas y siempre se pueden aprobechar las caracteristicas del mismo en los esquemas a realizar.

A veces complicar las matematicas nos brinda mejores resultados computacionales. Perder precisión para ganar velocidad
es una buena alternativa, particularmente si evitamos tener que resolver sistemas de ecuaciones ya que es un proceso
lento para matrices grandes y sin propiedades que nos ayuden, como en este laboratorio.

## Referencias
<div id='ref1'\> [1] [IPython Notebook Documentation](http://ipython.readthedocs.org/en/stable/) IPython Documentation. *desc*. Revisado 16/12/2015
<div id='ref1'\> [2] [Python Documentation](https://docs.python.org/3/) Python Documentation. *desc*. Revisado 16/12/2015

## Anexos
Comenzamos cargando los módulos necesarios:

In [ ]:
import numpy as np
import scipy as sp
from scipy.optimize import fsolve
from matplotlib import pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D
#from matplotlib.colors import cnames
from matplotlib import animation

#%matplotlib notebook
#%load_ext memory_profiler

<div id='an5'\> 
### Utilidades

In [ ]:
"""
> sol must be a 3D numpy array, with dimensions (Nt,Nx,Ny)
> time_step:  miliseconds to update canvas
> frame_step: Jump step for frames to show
"""
def animate_pattern(sol, time_step=10, frame_step=20):
    #time index
    t_ind = 0
    fig = plt.figure()
    im = plt.imshow(sol[t_ind], cmap=plt.cm.winter)
    plt.xticks([]); plt.yticks([])

    #setting the number of frames
    frames = int(sol.shape[0]/frame_step)
    
    #update function
    def updatefig(t_ind):
        im = plt.imshow(sol[t_ind*frame_step], cmap=plt.cm.winter)
        return im,
    
    #animate it!
    ani = animation.FuncAnimation(fig, updatefig, frames=frames, 
                    interval=time_step, blit=True, repeat=False)
    plt.show()